The main purpose of this dataset is to perform a multilaber classification of wheather a person will take **H1N1 Vaccine** or **Seasonal Flu Vaccine**.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


**Set up the configuration**

Let's start off with our Python program and begin with installing and importing the necessary packages.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

%matplotlib inline

In [ ]:
data_X = pd.read_csv(os.path.join(dirname,'training_set_features.csv'))
data_y = pd.read_csv(os.path.join(dirname,'training_set_labels.csv'))

In [ ]:
data_X.head(50)

In [ ]:
data_y.head()

In [ ]:
data_X.info()

In [ ]:
data_y.info()

In [ ]:
data_y['h1n1_vaccine'].value_counts()

In [ ]:
data_y['seasonal_vaccine'].value_counts()

In [ ]:
data_X.isna().sum()

We can see from above that we have lot of missing values for health insurance, employment_industry, employment_occuppation

In [ ]:
data_X.max()

In [ ]:
data_X = data_X.drop('respondent_id', axis = 1)

In [ ]:
plt.rcParams["figure.figsize"] = (15,30)

In [ ]:
data_X['health_worker'].sum()

In [ ]:
data_X['health_worker'].isna().sum()

In [ ]:
hw_with_insurance = data_X[(data_X['health_worker'] == 1) & (data_X['health_insurance']==1)]
hw_with_insurance['health_worker'].sum()

In [ ]:
hw_without_insurance = data_X[(data_X['health_worker'] == 1) & (data_X['health_insurance']==0)]
hw_without_insurance['health_worker'].sum()

Some important points
If health worker, we can assume that they have health insurance
Countries with universal healthcare include Austria, Belarus, Bulgaria, Croatia, Czech Republic, Denmark, Finland, France, Germany, Greece, Iceland, Isle of Man, Italy, Luxembourg, Malta, Moldova, Norway, Poland, Portugal, Romania, Russia, Serbia, Spain, Sweden, Switzerland, Turkey, Ukraine, and the United Kingdom --- Not valid as the data is only for US

In [ ]:
# close to half the values of these columns are NULL so discarding
data_X.drop(['health_insurance','employment_industry','employment_occupation'],axis=1,inplace = True)

In [ ]:
conc = [data_X, data_y]
data = pd.concat(conc, axis = 1)
data.head()
data = data.drop('respondent_id', axis =1)
data

Filling up all the null values with median values

In [ ]:
# For categorical features
for col in data.columns:
    if data[col].isnull().sum() and data[col].dtypes == 'object':
        data[col].loc[(data[col].isnull())] = data[col].mode().max()

In [ ]:
# For Numerical features
for col in data.columns:
    if data[col].isnull().sum() and data[col].dtypes != 'object':
        data[col].loc[(data[col].isnull())] = data[col].median()

In [ ]:
data.isnull().sum()

In [ ]:
data

In [ ]:
# One Hot Encoding for Categorical Features

cols = data.columns # Data Columns

num_cols = data._get_numeric_data().columns # Numerical Columns
data_num = data[num_cols]

cate_columns  = list(set(cols) - set(num_cols)) # Categorical Columns

data_one_hot_cat = pd.DataFrame()

for each in data[cate_columns]:
    dummies = pd.get_dummies(data[each], prefix=each, drop_first=False)
    data_one_hot_cat = pd.concat([data_one_hot_cat, dummies], axis=1)
    
data_one_hot = pd.concat([data_num, data_one_hot_cat], axis=1)

data_one_hot

In [ ]:
# histograms
data_one_hot[num_cols].hist()
plt.show()

In [ ]:
data_one_hot_cat.hist()
plt.show()

In [ ]:
# Correlation between feature
def corrl(data):
    corr = data.corr()
    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))
    heat_plot = sns.heatmap(corr, mask = mask, vmax=.3, center=0,
                square=True, linewidths=1, cbar_kws={"shrink": .5}, annot=True, fmt='.2f', cmap='Spectral', xticklabels=True , yticklabels=True )
    sns.despine()
    heat_plot.figure.set_size_inches(30,25)

    plt.show()
    
corrl(data_num)
corrl(data_one_hot_cat)

In [ ]:
sns.catplot('h1n1_vaccine',data=data_y,kind='count', height = 3.5)
sns.catplot('seasonal_vaccine',data=data_y,kind='count', height = 3.5)

In [ ]:
categorical = data_X.select_dtypes(include = 'object').columns
data_X[categorical].head()